In [2]:
import numpy as np
from matplotlib import pyplot as plt
from nuscenes.utils.geometry_utils import view_points
from nuscenes.utils.data_classes import LidarPointCloud, RadarPointCloud, Box
from pyquaternion import Quaternion
import itertools
import inference_utils
import json
from typing import Any

Axis = Any

def get_detection_box(record):
    return Box(record['translation'], record['size'], Quaternion(record['rotation']),
                   name=record['detection_name'], token=record['sample_token'])

def get_box(record):
    return Box(record['translation'], record['size'], Quaternion(record['rotation']),
                   name=record['category_name'], token=record['sample_token'])



def boxes_to_sensor(boxes, pose_record, cs_record):
    """
    Map boxes from global coordinates to the vehicle's sensor coordinate system.
    :param boxes: The boxes in global coordinates.
    :param pose_record: The pose record of the vehicle at the current timestamp.
    :param cs_record: The calibrated sensor record of the sensor.
    :return: The transformed boxes.
    """
    boxes_out = []
    for box in boxes:
        # Create Box instance.
        box = Box(box.center, box.wlh, Quaternion(box.orientation))

        # Move box to ego vehicle coord system.
        box.translate(-np.array(pose_record['translation']))
        box.rotate(Quaternion(pose_record['rotation']).inverse)

        #  Move box to sensor coord system.
        box.translate(-np.array(cs_record['translation']))
        box.rotate(Quaternion(cs_record['rotation']).inverse)

        boxes_out.append(box)

    return boxes_out



def visualize_sample(nusc,
                     sample_token: str,
                     gt_boxes,
                     pred_boxes,
                     scores,
                     nsweeps: int = 1,
                     conf_th: float = 0.15,
                     eval_range: float = 50,
                     verbose: bool = True,
                     savepath: str = None) -> None:
    """
    Visualizes a sample from BEV with annotations and detection results.
    :param nusc: NuScenes object.
    :param sample_token: The nuScenes sample token.
    :param gt_boxes: Ground truth boxes grouped by sample.
    :param pred_boxes: Prediction grouped by sample.
    :param nsweeps: Number of sweeps used for lidar visualization.
    :param conf_th: The confidence threshold used to filter negatives.
    :param eval_range: Range in meters beyond which boxes are ignored.
    :param verbose: Whether to print to stdout.
    :param savepath: If given, saves the the rendering here instead of displaying.
    """
    # Retrieve sensor & pose records.
    sample_rec = nusc.get('sample', sample_token)
    sd_record = nusc.get('sample_data', sample_rec['data']['LIDAR_TOP'])
    cs_record = nusc.get('calibrated_sensor', sd_record['calibrated_sensor_token'])
    pose_record = nusc.get('ego_pose', sd_record['ego_pose_token'])

    # Get boxes.
#     boxes_gt_global = gt_boxes[sample_token]
#     boxes_est_global = pred_boxes[sample_token]
    boxes_gt_global = gt_boxes
    boxes_est_global = pred_boxes

    # Map GT boxes to lidar.
    boxes_gt = boxes_to_sensor(boxes_gt_global, pose_record, cs_record)

    # Map EST boxes to lidar.
    boxes_est = boxes_to_sensor(boxes_est_global, pose_record, cs_record)

#     # Add scores to EST boxes.
#     for box_est, box_est_global in zip(boxes_est, boxes_est_global):
#         box_est.score = box_est_global.detection_score

    # Get point cloud in lidar frame.
    pc, _ = LidarPointCloud.from_file_multisweep(nusc, sample_rec, 'LIDAR_TOP', 'LIDAR_TOP', nsweeps=nsweeps)

    # Init axes.
    %matplotlib notebook
    _, ax = plt.subplots(1, 1, figsize=(9, 9))

    # Show point cloud.
    points = view_points(pc.points[:3, :], np.eye(4), normalize=False)
    dists = np.sqrt(np.sum(pc.points[:2, :] ** 2, axis=0))
    colors = np.minimum(1, dists / eval_range)
    ax.scatter(points[0, :], points[1, :], c=colors, s=4)

    # Show ego vehicle.
    ax.plot(0, 0, 'x', color='black')

    # Show GT boxes.
    for box in boxes_gt:
        box.render(ax, view=np.eye(4), colors=('g', 'g', 'g'), linewidth=2)

    # Show EST boxes.
    for (box,score) in zip(boxes_est,scores):
        # Show only predictions with a high score.
#         assert not np.isnan(box.score), 'Error: Box score cannot be NaN!'
        if score >= conf_th:
            box.render(ax, view=np.eye(4), colors=('b', 'b', 'b'), linewidth=1)

    # Limit visible range.
    axes_limit = eval_range + 3  # Slightly bigger to include boxes that extend beyond the range.
    ax.set_xlim(-axes_limit, axes_limit)
    ax.set_ylim(-axes_limit, axes_limit)
    # Show / save plot.
    if verbose:
        print('Rendering sample token %s' % sample_token)
    plt.title(sample_token)
    if savepath is not None:
        plt.savefig(savepath)
        plt.close()
    else:
        plt.show()

def visualize_prediction(nusc, model_pred, index=0,conf_th=0.15):
    nusc_annos = inference_utils.get_nusc_style(model_pred, nusc) # convert into sample annos style object
    sample_token, det_annotations_attrs = next(itertools.islice(nusc_annos['results'].items(), index, None)) # take a single frame
    scores = model_pred[index]['scores']
    
    det_boxes = [get_detection_box(det_annotation_attr) for det_annotation_attr in det_annotations_attrs] #convert annos-style object into Box object
    annotations_list =nusc.get('sample',sample_token)['anns']
    gt_boxes = [get_box(nusc.get('sample_annotation', anno)) for anno in annotations_list]
    
    visualize_sample(nusc, sample_token, gt_boxes=gt_boxes, pred_boxes=det_boxes, scores=scores, conf_th=conf_th, nsweeps=10) # plot

## visualizaing prediction from detection


# visualize_sample(nusc, sample_token, gt_boxes=gt_boxes, pred_boxes=det_boxes, nsweeps=10)
# visualize_prediction(nusc, predictions, 1)